# Downloading needed CMIP6 data

## Purpose of the notebook

This notebook aims at retrieving all the needed url(s) to download the needed CMIP6 data for a given analysis.

## Importations

In [ ]:
from pyesgf.search import SearchConnection
import os
import pandas as pd
import requests

## Criterias for the data research